In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns  

from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.preprocessing import RobustScaler # it is not affected by outliers.
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split, learning_curve
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
import xgboost as xgb
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import FunctionTransformer
from sklearn.base import TransformerMixin, BaseEstimator
from sklearn.semi_supervised import LabelSpreading

from sklearn.metrics import mean_absolute_percentage_error, mean_squared_error, r2_score

from sklearn.feature_selection import f_regression, SelectKBest, RFECV
# from imputer import create_full_pipeline
from imputer import create_full_pipeline, HotEncoderCategorical
from sklearn.ensemble import RandomForestClassifier
from sklearn.semi_supervised import SelfTrainingClassifier

data_original = pd.read_csv('Assets/Data/welddb.csv', delimiter='\s+', header=None)

data = data_original.copy().replace({"N": np.nan})

# Name the columns
data.columns = [
    'Carbon concentration (weight%)', 
    'Silicon concentration (weight%)', 
    'Manganese concentration (weight%)', 
    'Sulphur concentration (weight%)', 
    'Phosphorus concentration (weight%)', 
    'Nickel concentration (weight%)', 
    'Chromium concentration (weight%)', 
    'Molybdenum concentration (weight%)', 
    'Vanadium concentration (weight%)', 
    'Copper concentration (weight%)', 
    'Cobalt concentration (weight%)', 
    'Tungsten concentration (weight%)', 
    'Oxygen concentration (ppm by weight)', 
    'Titanium concentration (ppm by weight)', 
    'Nitrogen concentration (ppm by weight)', 
    'Aluminium concentration (ppm by weight)', 
    'Boron concentration (ppm by weight)', 
    'Niobium concentration (ppm by weight)', 
    'Tin concentration (ppm by weight)', 
    'Arsenic concentration (ppm by weight)', 
    'Antimony concentration (ppm by weight)', 
    'Current (A)', 
    'Voltage (V)', 
    'AC or DC', 
    'Electrode positive or negative', 
    'Heat input (kJ/mm)', 
    'Interpass temperature (°C)', 
    'Type of weld', 
    'Post weld heat treatment temperature (°C)', 
    'Post weld heat treatment time (hours)', 
    'Yield strength (MPa)', 
    'Ultimate tensile strength (MPa)', 
    'Elongation (%)', 
    'Reduction of Area (%)', 
    'Charpy temperature (°C)', 
    'Charpy impact toughness (J)', 
    'Hardness (kg/mm2)', 
    '50% FATT', 
    'Primary ferrite in microstructure (%)', 
    'Ferrite with second phase (%)', 
    'Acicular ferrite (%)', 
    'Martensite (%)', 
    'Ferrite with carbide aggregate (%)', 
    'Weld ID'
]

# Definição das colunas categóricas e numéricas
categoric_features = ['AC or DC', 'Electrode positive or negative', 'Type of weld']
numeric_features = ['Sulphur concentration (weight%)', 'Nickel concentration (weight%)', 
                    'Silicon concentration (weight%)', 'Phosphorus concentration (weight%)', 
                    'Titanium concentration (ppm by weight)', 'Nitrogen concentration (ppm by weight)', 
                    'Oxygen concentration (ppm by weight)', 'Voltage (V)', 'Heat input (kJ/mm)']

# Separação dos dados em treino e teste
X = data.drop(columns = ["Yield strength (MPa)", "Weld ID"])
y = data["Yield strength (MPa)"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Criando o pipeline com as features necessárias
percent_n_sorted = X_train.isna().mean().sort_values(ascending=False)
full_pipeline = create_full_pipeline()

# Dropping missing values in test
X_test_clean = X_test[~y_test.isna()]
y_test_clean = y_test.dropna().astype(float)

# Aplicando o pipeline nos dados de treino
X_train_transformed = X_train.copy()
X_train_transformed = full_pipeline.fit_transform(X_train_transformed)

# Aplicando o pipeline nos dados de teste
X_test_transformed = full_pipeline.transform(X_test_clean)

# hotencoder = HotEncoderCategorical(X_train_transformed)
# X_train_transformed = hotencoder.fit_transform(X_train_transformed)
# X_test_transformed = hotencoder.transform(X_test_transformed)

/Users/gabriellima/Documents/CentraleSupelec/Mention/ApprAutomatique/Project/Welding-Quality-Prediction/.venv/lib/python3.11/site-packages/sklearn/impute/_iterative.py:825: ConvergenceWarning: [IterativeImputer] Early stopping criterion not reached.
  warnings.warn(
/Users/gabriellima/Documents/CentraleSupelec/Mention/ApprAutomatique/Project/Welding-Quality-Prediction/imputer.py:51: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X[self.numeric_features] = X[self.numeric_features].apply(pd.to_numeric, errors='coerce')


# Preprocessing

In [3]:
# Criar um modelo base (nesse caso, um RandomForest)
# base_model = RandomForestClassifier()
base_model = LabelSpreading(kernel='knn', n_neighbors=7)

# Criar o modelo de auto-treinamento
self_training_model = SelfTrainingClassifier(base_model)

# X_train contém todas as variáveis explicativas
# y_train contém os labels, mas com valores NaN para os rótulos faltantes
# Importante: O SelfTrainingClassifier trata os valores faltantes como -1, então convertemos NaN para -1
y_train_imputed = y_train.fillna(-1)

# Treinar o modelo de auto-treinamento
self_training_model.fit(X_train_transformed, y_train_imputed)

y_train_imputed = y_train_imputed.astype(float)

# Agora, o modelo está treinado e os labels foram preenchidos (imputados)
y_train_completed = self_training_model.predict(X_train_transformed).astype(float)

In [7]:
pd.DataFrame(X_train_transformed)

,0,1,2,3,4,5,6,7,8,9,...,30,31,32,33,34,35,36,37,38,39
0,0.368186,0.755089,0.104193,-0.222542,-0.409326,0.402424,0.638008,-0.520152,-2.798737,0.755089,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
1,-1.021843,-0.082653,0.518459,0.564938,-0.425248,-0.554179,-1.489875,-0.520152,-0.145209,-0.082653,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.073331,-0.170836,0.337218,-0.316289,-0.425248,-0.554179,0.367590,-0.069517,-0.145209,-0.170836,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1.884581,-0.347203,0.673808,-0.211292,-0.401150,-0.541756,-1.489875,0.072789,2.508318,-0.347203,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
4,-0.095157,0.005531,0.492567,-0.316289,-0.425248,-0.554179,0.403055,-0.057658,-0.145209,0.005531,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1151,0.199697,-0.347203,0.337218,-0.256291,-0.412339,0.514235,0.505016,0.072789,2.508318,-0.347203,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
1152,0.368186,-0.259020,-1.708218,-0.248791,0.521478,2.054738,1.435965,-0.520152,-0.808591,-0.259020,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
1153,-1.232453,1.769198,1.398773,-0.297540,-0.386519,-0.554179,1.772880,0.428554,-0.145209,1.769198,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1154,0.073331,0.975548,1.062182,-0.316289,-0.425248,-0.554179,0.434086,-0.520152,1.181554,0.975548,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0


In [6]:
print(X_train_transformed.shape)
print(X_test_transformed.shape)

(1156, 40)
(244, 39)


In [4]:
# Categoric features 

## Linear Regression
def print_metrics(y_pred_lr, y_test_clean):
    print("MAPE :", mean_absolute_percentage_error(y_pred_lr, y_test_clean))
    print("R2 :", r2_score(y_pred_lr, y_test_clean))
    print("MSE :", mean_squared_error(y_pred_lr, y_test_clean))

def train_test_models(X_train_transformed, y_train_completed):

    print("\n -------------------- Linear Regression -------------------- \n")
    lr_pipeline = Pipeline(
        [
            ("Regressor", LinearRegression())
        ]
    )

    lr_pipeline.fit(X_train_transformed, y_train_completed)
    y_pred_lr = lr_pipeline.predict(X_test_transformed)
    print_metrics(y_pred_lr, y_test_clean)

    ## Ridge Regression

    print("\n -------------------- Ridge Regression -------------------- \n")
    ridge_pipeline = Pipeline(
        [
            ("Ridge Regressor", Ridge())
        ]
    )

    ridge_pipeline.fit(X_train_transformed, y_train_completed)
    y_pred_ridge = ridge_pipeline.predict(X_test_transformed)
    print_metrics(y_pred_ridge, y_test_clean)

    ## Lasso Regression

    print("\n -------------------- Lasso Regression -------------------- \n")
    lasso_pipeline = Pipeline(
        [
            ("Lasso Regressor", Lasso())
        ]
    )

    lasso_pipeline.fit(X_train_transformed, y_train_completed)
    y_pred_lasso = lasso_pipeline.predict(X_test_transformed)

    print_metrics(y_pred_lasso, y_test_clean)

    ## ElasticNet Regression

    print("\n -------------------- ElasticNet Regression -------------------- \n")
    ElasticNet_pipeline = Pipeline(
        [
            ("Regressor", ElasticNet())
        ]
    )

    ElasticNet_pipeline.fit(X_train_transformed, y_train_completed)
    y_pred_ElasticNet = ElasticNet_pipeline.predict(X_test_transformed)

    print_metrics(y_pred_ElasticNet, y_test_clean)

    ## Decision Tree Regression

    print("\n -------------------- Decision Tree Regression -------------------- \n")
    tree_pipeline = Pipeline(
        [
            ("Regressor", DecisionTreeRegressor())
        ]
    )

    tree_pipeline.fit(X_train_transformed, y_train_completed)
    y_pred_tree = tree_pipeline.predict(X_test_transformed)

    print_metrics(y_pred_tree, y_test_clean)

    ## Random Forest Regression

    print("\n -------------------- Random Forest Regression -------------------- \n")
    RF_pipeline = Pipeline(
        [
            ("Regressor", RandomForestRegressor())
        ]
    )

    RF_pipeline.fit(X_train_transformed, y_train_completed)
    y_pred_RF = RF_pipeline.predict(X_test_transformed)

    print_metrics(y_pred_RF, y_test_clean)

    ## Gradient Boosting Regression

    print("\n -------------------- Gradient Boosting Regression -------------------- \n")
    gb_pipeline = Pipeline(
        [
            ("Regressor", GradientBoostingRegressor())
        ]
    )

    gb_pipeline.fit(X_train_transformed, y_train_completed)
    y_pred_gb = gb_pipeline.predict(X_test_transformed)

    print_metrics(y_pred_gb, y_test_clean)

train_test_models(X_train_transformed, y_train_completed)


 -------------------- Linear Regression -------------------- 



ValueError: X has 39 features, but LinearRegression is expecting 40 features as input.

In [ ]:
# Treinando o modelo XGBoost
xbg_pipeline = Pipeline(
        [
            ("XGB", xgb.XGBRegressor(objective='reg:squarederror', random_state=42))
        ]
    )
xbg_pipeline.fit(X_train_transformed, y_train_completed)

y_pred_xgb = xbg_pipeline.predict(X_test_transformed)

print_metrics(y_pred_xgb, y_test_clean)